In [1]:
import openai
import os
import IPython
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.callbacks import get_openai_callback

from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# explicitly set the API key for embedding
import openai
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']

openai_api_base = os.environ['OPENAI_API_BASE']
azure_development_name = os.environ['AZURE_DEVELOPMENT_NAME']
openai_api_key = os.environ['OPENAI_API_KEY']

In [3]:
# setup model in Azure Openai, and the model used to extract information from vectorstore

model = AzureChatOpenAI(
    openai_api_base=openai_api_base,
    openai_api_version="2023-03-15-preview",
    deployment_name=azure_development_name,
    openai_api_key=openai_api_key,
    openai_api_type='azure',
    temperature=0.2,
)

# model using to extract information from vectorstore
llm = AzureChatOpenAI(
    openai_api_base=openai_api_base,
    openai_api_version="2023-03-15-preview",
    deployment_name=azure_development_name,
    openai_api_key=openai_api_key,
    openai_api_type='azure',
    temperature=0,
    max_tokens=256
)

In [4]:
"""import pdfminer"""
from langchain.document_loaders import ReadTheDocsLoader, PyMuPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS


loader = PyMuPDFLoader("upload/Resume_Jesse_Chow.pdf")
raw_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(raw_documents)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)
db = FAISS.from_documents(documents, embeddings)
resume_retriever = db.as_retriever()

In [5]:
# create job duties 
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document

jd = """

Job duties of this Role:

At the core of Accenture's Technology business, you are eager to learn and use that curiosity to solve technology problems through developing, designing and maintaining software products or systems that enable client strategies to improve the way our clients and the world works.

You will be using your versatility and experience to create and support technology solutions that meet client requirements from analysis to implementation. The latest SDLC best practices will be applied to continuously improve the quality and efficiency of the Accenture development teams.

Come JOIN US if you have:
• Hands-on experience in application development and design on web applications that integrate to large scale/ mission critical systems using different technologies
• Strong in Web (Responsive UI, Angular, React, JavaScript, HTML, CSS) Development
• Knowledge on Redux and Saga is a plus
• Interested to explore other technologies integration
• Passionate and keen to develop your profession in technical delivery, and strive to deliver the best design, codes and practice
• Excellent communication skill and the ability to interact professionally with diverse group of stakeholders, internally and externally
• University Degree in Computer Science/Engineering, Information Technology/System, or other relevant disciplines desirable
• High proficiency in both verbal and written English and Cantonese (Mandarin is an added advantage)
• Dynamic and adaptive to the global collaborative project team environment
• Candidates with more related experience can be considered as senior position"""


jd_doc = Document(page_content=jd)
split_doc = text_splitter.split_documents([jd_doc])
job_db = FAISS.from_documents(split_doc, embeddings)
job_retriever = job_db.as_retriever()

In [6]:
"""logging """
import logging

logging.getLogger("openai").setLevel(logging.DEBUG) # logging.INFO or logging.DEBUG

In [7]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# memory = ConversationBufferWindowMemory(
#     memory_key="chat_history",  # important to align with agent prompt (below)
#     k=10,
#     return_messages=True
# )

readonly_memory = ReadOnlySharedMemory(memory=memory)

In [8]:
template = """This is a conversation between a candidate and a recruiter. The candidate is applying for the job with resume provided.

{chat_history}

Evaluate the performance of candidate from conversation below. 

{input}:
"""

rating_prompt = PromptTemplate(
    input_variables=["input", "chat_history"], 
    template=template
)

rating_chain = LLMChain(
    llm=llm,
    prompt=rating_prompt,
    verbose=True,
    memory=readonly_memory,
)

In [9]:
from langchain.chains import RetrievalQA

resume_prompt_template = """Use the following pieces of context to provide information from the resume of the candidate to help rewrite the question to better assess the candidate.

{context}

Question: {question}
Answer:"""

job_prompt_template = """Use the following pieces of context to provide information of the job roles and duties to help rewrite the question to assess whether the candidate meets requirements included in the context.

{context}

Question: {question}
Answer:"""


RESUME_PROMPT = PromptTemplate(
    template=resume_prompt_template, input_variables=["context", "question"]
)

JOB_PROMPT = PromptTemplate(
    template=job_prompt_template, input_variables=["context", "question"]
)

resume_chain_type_kwargs = {"prompt": RESUME_PROMPT}
job_chain_type_kwargs = {"prompt": JOB_PROMPT}

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=resume_retriever,
    chain_type_kwargs=resume_chain_type_kwargs
)

jd_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=job_retriever,
    chain_type_kwargs=job_chain_type_kwargs
)

In [10]:
"""create tool"""
from langchain.agents import Tool

tool_desc = """Use this tool to create interview question related to skills and experience of the candidate. This tool can also be used for create follow up questions from candidate answers."""

jd_desc = """Use this tool to provide information of the job position which the candidate applied to create interview question. This tool can also be used for create follow up questions from candidate answers."""

rating_desc = """Useful for when you summarize a conversation after interview finished. The input to this tool should be a string. Please summarize whether the candidate is a good fit for the position in the following 5 assessment criteria: 1. Technical ability 2. Leadership skills 3. Interpersonal/team skills 4. Presentation skills 5. Attitude and rate the candidate in a scale of 1 to 5, 1 is lowest and 5 is highest."""

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name="resume"
), Tool(
    func=jd_retriever.run,
    description=jd_desc,
    name="job_duties"
), Tool(
    func=rating_chain.run,
    description=rating_desc,
    name="rating"
)]

In [11]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    tools=tools, 
    llm=model,
    agent='chat-conversational-react-description',
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)

In [12]:
from langchain.agents import initialize_agent    
# change to 'generate' to ensure meaningful responses
evaluate_agent = initialize_agent(
    tools=tools, 
    llm=model,
    agent='zero-shot-react-description',
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate", 
    memory=memory,
)

In [14]:
""" modify prompt for interviewing bot"""

sys_msg = """You are a friendly interviewer asking question to a candidate who applied for the job position in a job interview. Your job is to find out whether the candidate is suitable for the job position.

Instructions:
- Count the number of questions you asked
- You can only ask 5 questions to the candidate
- You should keep count of the number of questions you asked, and should not tell the candidate how many questions you asked
- After 5 question, you should summarize the performance of the candidate
- You should not ask question that is not related to the job roles and the resume
- You should only give 1 sentence feedback to previous answer and ask then ask question
- You should not answer any questions from candidate
- You must not tell the candidate any information about any assessment criteria and your assessment in the middle of the interview
- Start asking candidate to introduce themselves to start the interview, this is the first question you should ask
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

In [15]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='You are a friendly interviewer asking question to a candidate who applied for the job position in a job interview. Your job is to find out whether the candidate is suitable for the job position.\n\nInstructions:\n- Count the number of questions you asked\n- You can only ask 5 questions to the candidate\n- You should keep count of the number of questions you asked, and should not tell the candidate how many questions you asked\n- After 5 question, you should summarize the performance of the candidate\n- You should not ask question that is not related to the job roles and the resume\n- You should only give 1 sentence feedback to previous answer and ask then ask question\n- You should not answer any questions from candidate\n- You must n

In [16]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[2].prompt.template
)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> resume: Use this tool to create interview question related to skills and experience of the candidate. This tool can also be used for create follow up questions from candidate answers.
> job_duties: Use this tool to provide information of the job position which the candidate applied to create interview question. This tool can also be used for create follow up questions from candidate answers.
> rating: Useful for when you summarize a conversation after interview finished. The input to this tool should be a string. Please summarize whether the candidate is a good fit for the position in the following 5 assessment criteria: 1. Technical ability 2. Leadership skills 3. Interpersonal/team skills 4. Presentation skills 5. Attitude and rate the candidate in a scale of 1 to 5, 1 is lowest and 5 is highest.

RESPONSE FORMAT

In [12]:
import redis

endpoint = 'https://redis--jbcbh4t.happyhill-5bfa4661.eastus.azurecontainerapps.io'

r = redis.StrictRedis(host='redis.happyhill-5bfa4661.eastus.azurecontainerapps.io', port=6379)
r.ping()

TimeoutError: Timeout connecting to server